In [ ]:
import polars as pl
import networkx as nx
from math import sqrt
import time
import random
import rtsvg
rt = rtsvg.RACETrack()
from polars_spring_layout_opt import PolarsSpringLayoutOpt
# Create a graph
use_real_data = False
if use_real_data == False:
    from linknode_graph_patterns import LinkNodeGraphPatterns
    _patterns_ = LinkNodeGraphPatterns()
    g          = _patterns_.createPattern('mesh')
else:
    _base_network_ = '0'
    _base_dir_     = '../../data/stanford/facebook/'
    _layout_file_  = _base_dir_ + _base_network_ + '.layout.parquet'
    _edges_ = open(_base_dir_ + _base_network_ + '.edges', 'rt').read()
    _lu_ = {'fm':[], 'to':[]}
    for _edge_ in _edges_.split('\n'):
        if _edge_ == '': continue
        _lu_['fm'].append(_edge_.split(' ')[0])
        _lu_['to'].append(_edge_.split(' ')[1])
    df = pl.DataFrame(_lu_)
    g            = rt.createNetworkXGraph(df, [('fm', 'to')])

    # Keep the largest connected component
    components = list(nx.connected_components(g))
    subgraphs = [g.subgraph(c).copy() for c in components]
    g = subgraphs[0]
    for i in range(len(subgraphs)):
        if len(subgraphs[i].nodes()) > len(g.nodes()): g = subgraphs[i]

# Create the dataframe version of the graph
_lu_ = {'fm':[], 'to':[]}
for edge in g.edges: _lu_['fm'].append(edge[0]), _lu_['to'].append(edge[1])
df = pl.DataFrame(_lu_)
print(f'edges: {len(g.edges)} | nodes: {len(g.nodes)}')

In [ ]:
#
# "Graph Drawing by Force-directed Placement"
# Fruchterman & Reingold
# Software -- Practice and Experience, Vol. 21 (1 1), 1129-1164 (November 1991)
#
# Figure 1 (Force-directed Placement)
# - with the optimization to only consider vertices w/in 2*k of the vertex for repulsion
#
pos_history = {}
for _node_ in g.nodes: pos_history[_node_] = []

t    = 5.0
W, H = 256, 256
pos  = {}
for _node_ in g.nodes(): pos[_node_] = random.random()*W, random.random()*H
area = W*H
C   = 0.2 # paper says that C is found experimentally...
k   = C*sqrt(area/len(g.nodes()))
def f_a(z): return  z**2 / k             # pseudocode says "x" vs "z" ... paper comments say "z"
def f_r(z): return  ((k**2)/z)*(2*k - z) # updated formula when only using near neighbors for repulsion

_iterations_ = 2*len(g.nodes())
for i in range(_iterations_):
    for _node_ in g.nodes: pos_history[_node_].append(pos[_node_])
    disp = {}
    # calculate repulsive forces (primary difference in the optimized version)
    for v in g.nodes():
        disp[v] = 0.0, 0.0
        for u in g.nodes():
            if u == v or g.has_edge(u, v): continue
            _diff_     = pos[v][0] - pos[u][0], pos[v][1] - pos[u][1]
            _diff_len_ = max(sqrt(_diff_[0]**2 + _diff_[1]**2), 0.001)
            if _diff_len_ > 2*k: continue
            disp[v]    = disp[v][0] + (_diff_[0]/_diff_len_) * f_r(_diff_len_), disp[v][1] + (_diff_[1]/_diff_len_) * f_r(_diff_len_)
    # calculate the attractive forces
    for e in g.edges():
        e_v, e_u, w = e[0], e[1], 1 if 'weight' not in g[e[0]][e[1]] else g[e[0]][e[1]]['weight']
        _diff_     = pos[e_v][0] - pos[e_u][0], pos[e_v][1] - pos[e_u][1]
        _diff_len_ = max(sqrt(_diff_[0]**2 + _diff_[1]**2), 0.001)
        disp[e_v]  = disp[e_v][0] - (_diff_[0]/(_diff_len_*w)) * f_a(_diff_len_), disp[e_v][1] - (_diff_[1]/(_diff_len_*w)) * f_a(_diff_len_)
        disp[e_u]  = disp[e_u][0] + (_diff_[0]/(_diff_len_*w)) * f_a(_diff_len_), disp[e_u][1] + (_diff_[1]/(_diff_len_*w)) * f_a(_diff_len_)
    # limit the maximum displacement to the temperature t
    # ... and then prevent from being displaced outside frame
    for v in g.nodes():
        disp_len   = max(sqrt(disp[v][0]**2 + disp[v][1]**2), 0.001)
        xmag, ymag = t if abs(disp[v][0]) > t else disp[v][0], t if abs(disp[v][1]) > t else disp[v][1]
        pos[v] = pos[v][0] + (disp[v][0]/disp_len)*xmag, pos[v][1] + (disp[v][1]/disp_len)*ymag
    t *= 0.999 # t = cool(t)

    # Rescale th positions to fit the frame
    # ... the paper has a several proposals for how to keep all the vertices in the frame
    # ... kindof unsure if this is necessary or not
    x0, y0, x1, y1 = pos[v][0], pos[v][1], pos[v][0], pos[v][1]
    for _node_ in pos.keys():
        x0, y0 = min(x0, pos[_node_][0]), min(y0, pos[_node_][1])
        x1, y1 = max(x1, pos[_node_][0]), max(y1, pos[_node_][1])
    for _node_ in pos.keys(): pos[_node_] = W*(pos[_node_][0] - x0)/(x1-x0), H*(pos[_node_][1] - y0)/(y1-y0)

for _node_ in g.nodes: pos_history[_node_].append(pos[_node_])

x0, y0, x1, y1 = pos[v][0], pos[v][1], pos[v][0], pos[v][1]
for k in pos_history.keys():
    x0, y0 = min(x0, min(pos_history[k][0])), min(y0, min(pos_history[k][1]))
    x1, y1 = max(x1, max(pos_history[k][0])), max(y1, max(pos_history[k][1]))

# Convert it to an animation
_dur_ = '2s'
svg = [f'<svg x="0" y="0" width="512" height="512" viewBox="{x0} {y0} {x1-x0} {y1-y0}">']
svg.append(f'<rect x="{x0}" y="{y0}"  width="{x1-x0}" height="{y1-y0}" fill="white" />')
for _edge_ in g.edges:
    u, v = _edge_[0], _edge_[1]
    svg.append(f'<line x1="{pos[u][0]}" y1="{pos[u][1]}" x2="{pos[v][0]}" y2="{pos[v][1]}" stroke="black" stroke-width="0.2">')
    svg.append(f'<animate attributeName="x1" values="{";".join([f"{_pos_[0]}" for _pos_ in pos_history[u]])}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append(f'<animate attributeName="y1" values="{";".join([f"{_pos_[1]}" for _pos_ in pos_history[u]])}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append(f'<animate attributeName="x2" values="{";".join([f"{_pos_[0]}" for _pos_ in pos_history[v]])}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append(f'<animate attributeName="y2" values="{";".join([f"{_pos_[1]}" for _pos_ in pos_history[v]])}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append('</line>')
for _node_ in g.nodes:
    svg.append(f'<circle cx="{pos_history[_node_][0][0]}" cy="{pos_history[_node_][0][1]}" r="1.2" fill="blue">')
    svg.append(f'<animate attributeName="cx" values="{";".join([f"{_pos_[0]}" for _pos_ in pos_history[_node_]])}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append(f'<animate attributeName="cy" values="{";".join([f"{_pos_[1]}" for _pos_ in pos_history[_node_]])}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append('</circle>')
svg.append('</svg>')

_lu_ = {'fm':[], 'to':[]}
for edge in g.edges: _lu_['fm'].append(edge[0]), _lu_['to'].append(edge[1])
df = pl.DataFrame(_lu_)
rt.tile([''.join(svg), rt.link(df, [('fm','to')],pos)], spacer=10)

In [ ]:
_pslo_ = PolarsSpringLayoutOpt(g)
_psl_  = rtsvg.PolarsSpringLayout(g) 
rt.tile([rt.link(df, [('fm','to')], _pslo_.results()),
         rt.link(df, [('fm','to')], _psl_.results())], spacer=10)

In [ ]:
print(f'{" "*30}  Opt    Non-Opt')
for _part_ in _pslo_.time_lu: print(f"{_part_:<30}  {_pslo_.time_lu[_part_]:0.3f}  {_psl_.time_lu[_part_]:0.3f}")
#                                 Opt    Non-Opt
# dist_df_create                  0.003  0.003
# pos_df_create                   0.000  0.000
# cross_join_iteration            0.576  0.585
# repulse_iteration               4.084  1.437 <-- 3.5x longer for the repulse portion in the "optimized" version ...
# attract_iteration               1.076  0.884
# sum_iteration                   0.841  0.918
# adjust_iteration                0.572  0.465
# copy_pos                        0.001  0.001

In [ ]:
_df_opt_ = pl.DataFrame({'rows':_pslo_.repulse_rows}).with_row_index().rename({'index':'iteration'}).with_columns(pl.lit('opt').alias('algorithm'))
_df_reg_ = pl.DataFrame({'rows':_psl_ .repulse_rows}).with_row_index().rename({'index':'iteration'}).with_columns(pl.lit('reg').alias('algorithm'))
_df_     = pl.concat([_df_opt_, _df_reg_])
rt.tile([rt.histogram(_df_, bin_by='algorithm', count_by='rows', color_by='algorithm'),
         rt.xy(_df_, x_field='iteration', y_field='rows', dot_size=None, line_groupby_field='algorithm', line_groupby_w=2.0, w=1024, color_by='algorithm')])

In [ ]:
# never finished after minutes .. need to go look at the implementation
#rt.tile([rt.graphLayoutSVGAnimation(_pslo_.pos_history, g, w=512, h=512, r=0.2), 
#         rt.graphLayoutSVGAnimation(_psl_ .pos_history, g, w=512, h=512, r=0.2)])

In [ ]:
_tiles_ = []
for i in range(len(_patterns_)):
    _g_ = _patterns_.createPattern(_patterns_[i])

    _lu_ = {'fm':[], 'to':[]}
    for edge in _g_.edges: _lu_['fm'].append(edge[0]), _lu_['to'].append(edge[1])
    _df_ = pl.DataFrame(_lu_)

    t0 = time.time()
    _pslo_ = PolarsSpringLayoutOpt(_g_)
    t1 = time.time()
    _tiles_.append(rt.titleSVG(rt.link(_df_, [('fm','to')], _pslo_.results()), f'{_patterns_[i]} O {t1-t0:0.3f}'))

    t2 = time.time()
    _psl_  = rtsvg.PolarsSpringLayout(_g_) 
    t3 = time.time()
    _tiles_.append(rt.titleSVG(rt.link(_df_, [('fm','to')],  _psl_.results()), f'{_patterns_[i]} R {t3-t2:0.3f}'))

rt.table(_tiles_, per_row=4)